In [123]:
import numpy as np
import string

# 音楽理論の参考サイト https://watanabejunya.com/
class DiatonicKeys:
    def __init__(self):
        self.select_keys = ["C", "C#/D♭", "D", "D#/E♭", "E", "F", "F#/G♭", "G", "G#/A♭", "A", "A#/B♭", "B"]
        self._flat_keys = ["C", "D♭", "D", "E♭", "E", "F", "G♭", "G", "A♭", "A", "B♭", "B"]
        self._sharp_keys = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
        self._keys = [self._flat_keys, self._sharp_keys]
        # self.italy_keys = ["ド", "レ♭", "レ", "ミ♭", "ミ", "ファ", "ソ♭", "ソ", "ラ♭", "ラ", "シ♭", "シ"]
        
      
    def _change_pattern(self):
        """
        主Keyを先頭にしてリスト循環させる
        """
        return [np.roll(k, len(k)-self.target_index) for k in self._keys]

    def _select_scale(self):
        """
        numpy filtaに使うbool配列を返す
        """
        if self.scale_type == "major":
            # [全、全、半、全、全、全、半]
            interval = [1, 0, 1, 0 , 1, 1, 0, 1, 0, 1, 0, 1]
        elif self.scale_type == "natural_minor":
            # [全、半、全、全、半、全、全]
            interval = [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0]
        elif self.scale_type == "harmonic_minor":
            interval = [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1]
        elif self.scale_type == "melodic_minor":
            interval = [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]
        else:
            print("存在しないスケールが指定されています")
            return 
            
        return np.array([True if i==1 else False for i in interval], dtype=bool)
    
    def _jadge_unique_key(self):
        """
        白鍵キーが7つ存在している方を返す。なければ♭表記で返す
        """
        for keys in self.pickup_keys:
            if len({k[0] for k in keys}) == 7:
                return keys
        else:
            return self.pickup_keys[1]
    
    def daitonic(self, target_key="C", scale_type="major"):
        """
        ダイアトニック構成音のリストを返す
            args:
                [0]=key: str
                [1]=scale_type: str
            return: -> list
                [0]=flat_keys [♭]
                [1]=sharp_keys [#]
        """
        self.target_key = target_key
        self.scale_type = scale_type
        
        # 主キーのインデックスを取りリスト循環させる
        self.target_index = self.select_keys.index(self.target_key)       
        changed_keys = self._change_pattern()
        
        # 必要なキーのみ残しダイアトニック構成音を返す
        filter_scale = self._select_scale()
        self.pickup_keys = [k[filter_scale] for k in changed_keys]
        self.diatonic_keys = self._jadge_unique_key()
        return self.diatonic_keys
        
class MajorScale:
    def __init__(self):
        self._d = DiatonicKeys()
        
class MinorScale:
    def __init__(self):
        self._d = DiatonicKeys()
    
d = DiatonicKeys()
# d.daitonic("C#/D♭", "harmonic_minor")
anm = d.daitonic("A", "melodic_minor")


In [202]:
class BasicDiatonicChordConfig:
    def __init__(self):
        self.__scale_filter = None
        self.__tonic = None
        self.__dominant = None
        self.__sub_dominant = None
        self.__triad = None
        self.__tetrad = None
        
    @property
    def scale_filter(self):
        # 構成音として使うものを 1, そうでないものを 0 としたリストを格納する。 len=7/12音
        return self.__scale_filter
    @property
    def tonic(self):
        # index番号を格納する
        return self.__tonic
    @property
    def dominant(self):
        # index番号を格納する
        return self.__dominant
    @property
    def sub_dominant(self):
        # index番号を格納する
        return self.__sub_dominant
    @property
    def triad(self):
        # 三和音の場合の付属
        return self.__triad
    @property
    def tetrad(self):
        # 四和音の場合の付属
        return self.__tetrad
    
    def set_parameter(self):
        """
        propertyの値を各クラスの値で更新する
        """
        self.scale_filter()
        self.scale_filter = np.array(self.scale_filter, dtype=bool)
        self.tonic()
        self.dominant()
        self.sub_dominant()
        self.triad()
        self.tetrad()
    
    
class MajorScale(BasicDiatonicChordConfig):
    def __init__(self):
        super().__init__()
        self.set_parameter()

    def scale_filter(self):
        self.scale_filter = [1, 0, 1, 0 , 1, 1, 0, 1, 0, 1, 0, 1]
    def tonic(self):
        # I, Ⅱ, Ⅵ　
        self.tonic = [0, 2, 5]
    def dominant(self):
        self.dominant = [4, 6]
    def sub_dominant(self):
        self.sub_dominant = [3, 5]
    def triad(self):
        self.triad = ["", "m", "m", "", "", "m", "m(♭5)"]
    def tetrad(self):
        self.tetrad = ["M7", "m7", "m7", "M7", "7", "m7", "m7(♭5)"]
     
    
class NaturalMinorScale(BasicDiatonicChordConfig):
    def __init__(self):
        super().__init__()
        self.set_parameter()

    def scale_filter(self):
        self.scale_filter = [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0]
    def tonic(self):
        self.tonic = [0, 2]
    def dominant(self):
        self.dominant = [4, 6]
    def sub_dominant(self):
        self.sub_dominant = [1, 3, 5]
    def triad(self):
        self.triad = ["m", "m(♭5)", "", "m", "m", "", ""]
    def tetrad(self):
        self.tetrad = ["m7", "m7(♭5)", "M7", "m7", "m7", "M7", "7"]
        
        
class HarmonicMinorScale(BasicDiatonicChordConfig):
    """
    ナチュラルマイナーから七度が半音上がる
    """
    def __init__(self):
        super().__init__()
        self.set_parameter()
        
    def scale_filter(self):
        self.scale_filter = [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1]
    def tonic(self):
        self.tonic = [0, 2]
    def dominant(self):
        self.dominant = [4, 6]
    def sub_dominant(self):
        self.sub_dominant = [1, 3, 5]
    def triad(self):
        self.triad = ["m", "m(♭5)", "aug", "m", "", "", "dim"]
    def tetrad(self):
        self.tetrad = ["mM7", "m7(♭5)", "augM7", "m7", "7", "M7", "dim7"]
        
        
class MelodicMinorScale(BasicDiatonicChordConfig):
    """
    メージャースケールと比較して三度が半音上がる
    """
    def __init__(self):
        super().__init__()
        self.set_parameter()
        
    def scale_filter(self):
        self.scale_filter = [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]
    def tonic(self):
        self.tonic = [0, 2, 5]
    def dominant(self):
        self.dominant = [3, 4, 6]
    def sub_dominant(self):
        self.sub_dominant = [1]
    def triad(self):
        self.triad = ["m", "m", "aug", "", "", "m(♭5)", "m(♭5)"]
    def tetrad(self):
        self.tetrad = ["mM7", "m7", "augM7", "7", "7", "m7(♭5)", "m7(♭5)"]
        
        
class Scales:
    def __init__(self):
        self.scale_list = {
            "Major": MajorScale(),
            "Minor": NaturalMinorScale(),
            "HarmonicMinor": HarmonicMinorScale(),
            "MelodicMinor": MelodicMinorScale(),
        }
    def __call__(self):
        return self.scale_list

        
m= MajorScale()
mi = NaturalMinorScale()
mm = MelodicMinorScale()
print(m.scale_filter)
m.triad



[ True False  True False  True  True False  True False  True False  True]


['', 'm', 'm', '', '', 'm', 'm(♭5)']

In [201]:
class GenerateDiatoincChord:
    def __init__(self, main_key="C", scale_type="Major"):
        self.select_keys = ["C", "C#/D♭", "D", "D#/E♭", "E", "F", "F#/G♭", "G", "G#/A♭", "A", "A#/B♭", "B"]
        self._flat_keys = ["C", "D♭", "D", "E♭", "E", "F", "G♭", "G", "A♭", "A", "B♭", "B"]
        self._sharp_keys = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
        self.keys = [self._flat_keys, self._sharp_keys]
        
        self.main_key = main_key
    
    def circulate_keys(self, keys, target_key):
        """
        ターゲットKeyを先頭にしてリスト循環させる
        """
        target_index = keys.index(target_key)
        return np.roll(keys, len(keys)-target_index) 
    
    def _create_new_keys(self)
    
    def relative_key(self):
        pass
    
